<a href="https://colab.research.google.com/github/detsutut/AraucanaXAI/blob/master/araucana_XAI_special_issue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Araucana XAI Special Issue Notebook**

## **Init**

In [ ]:
%%capture
!pip3 install shap
!pip3 install araucanaxai
!pip3 install lime

In [ ]:
%%capture
from google.colab import files
import numpy as np
from itertools import combinations
import pandas as pd
import io
import time
import warnings
from tqdm import tqdm
from matplotlib import pyplot as plt
from IPython.display import display
import collections
import random

import araucanaxai
import shap
import lime

from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import precision_recall_curve, f1_score

# still have to fix this (the warning is just annoying but does not affect the pipeline in any way)
warnings.filterwarnings("ignore", message="X does not have valid feature names, but LogisticRegression was fitted with feature names")
warnings.filterwarnings("ignore", message="X does not have valid feature names, but RandomForestClassifier was fitted with feature names")
warnings.filterwarnings("ignore", message="X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names")
warnings.filterwarnings("ignore", message="X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names")
warnings.filterwarnings("ignore", message="X does not have valid feature names, but MLPClassifier was fitted with feature names")

## **Data**

In [ ]:
#UPLOAD DATA
uploaded = files.upload()

### MIMIC Dataset

In [ ]:
#MIMIC PREPROCESSING
df_raw = pd.read_csv(io.BytesIO(uploaded['MIMIC_III_dataset.csv']))
#df_raw = df_raw.groupby('In-hospital_death', group_keys=False).apply(lambda x: x.sample(frac=0.1))  # ********* <<< DEBUG, remove this row >>> **********

# Removal of features containing more than 90% of NaN
original_shape = df_raw.shape
print("DataFrame shape before NaN removal:", original_shape)
df = df_raw.dropna(thresh=round(df_raw.shape[0] * 0.90), axis=1)

# NaN removal by removing rows with at least one NaN value
# Check for NaN values
if df.isnull().any().any():  # If there's at least one column with NaN, the output is "True"
    old_row_size = original_shape[0]
    df = df.dropna(axis=0, how='any')
    print("DataFrame shape after NaN removal:", df.shape)
    new_row_size = df.shape[0]
    print("Relative percentage of removed rows (wrt the old row size): %.2f" % (
            (old_row_size - new_row_size) / old_row_size * 100))

# NaN values have now been removed, so the output should be "False"
print("Is there any NaN after NaN removal?", df.isnull().any().any())
df = df.drop(columns=["recordid"])

In [ ]:
#DATA PREPARATION
pred_col = 'In-hospital_death' #edit this line when using another dataset

X = df
y = X[pred_col]
X_feat = X.drop(columns=[pred_col])
feature_names = X_feat.columns

#check which features has less than 10 unique values to identify a subset of probable categorical features
print(feature_names[np.where([len(np.unique(X_feat.iloc[:,i]))<10 for i in range(X_feat.shape[1])])])
categorical_feat = ['CCU', 'CSU', 'SICU'] # edit this line when using another dataset
iscat = [x in categorical_feat for x in feature_names]

X_train, X_test, y_train, y_test = train_test_split(X_feat, y, test_size=0.33, random_state=42)

X_test_nd = X_test.to_numpy(dtype=float)
X_train_nd = X_train.to_numpy(dtype=float)

# Normalizing data ********* <<< DEBUG, remove next rows if you don't want normalization >>> **********
scaler = MinMaxScaler()

X_train_nd = scaler.fit_transform(X_train_nd)
X_train = pd.DataFrame(X_train_nd, columns=feature_names)

X_test_nd = scaler.fit_transform(X_test_nd)
X_test = pd.DataFrame(X_test_nd, columns=feature_names)

### HEPAR Dataset

In [ ]:
df_raw = pd.read_csv(io.BytesIO(uploaded['HEPAR_simulated_patients.csv']))
#df_raw = pd.read_csv(io.BytesIO(uploaded['HEPAR_simulated_patients_ood.tsv']))
df_raw.drop("Unnamed: 0", axis=1, inplace=True)
display(df_raw)

In [ ]:
#Enea
df = df_raw

# Categorical variables handled
df_num = df.copy()
"""Handling categorical and nominal data"""

nominal_feat = ['age', 'triglycerides', 'bilirubin', 'phosphatase', 'proteins', 'platelet', 
                'inr', 'urea', 'ESR', 'alt', 'ast','amylase', 'ggtp', 'cholesterol', 'albumin']

# preprocessing: data transformation
nominal_dict = {}
for n in nominal_feat:
  unique_val = df[n].unique().tolist()
  unique_num = [int(x.split('_')[1]) for x in unique_val]
  val2num = dict(zip(unique_val, unique_num))
  num2cat = dict(zip(sorted(unique_num), range(1, len(unique_num)+1)))
  dict_n = {}
  for k,v in val2num.items():
    dict_n[k] = num2cat[v]
  nominal_dict[n] = dict_n

special_feat = ['ChHepatitis', 'sex', 'Cirrhosis']

dict_chhepa = {'absent':0, 'active':1, 'persistent':2}
dict_sex = {'female':1, 'male':2}
dict_cirr = {'absent':0, 'decompensate':1, 'compensate':2}

df_num['sex'] = [dict_sex[x] for x in df['sex']]
df_num['ChHepatitis'] = [dict_chhepa[x] for x in df['ChHepatitis']]
df_num['Cirrhosis'] = [dict_cirr[x] for x in df['Cirrhosis']]

categorical_feat = [x for x in df.columns if x not in nominal_feat+special_feat]
dict_cat = {'absent':0, 'present':1}

#print(categorical_feat)
for c in categorical_feat:
  #print(c)
  df_num[c] = [dict_cat[x] for x in df[c]]

for n in nominal_feat:
  newcol = [nominal_dict[n][x] for x in df[n]]
  df_num[n] = newcol

In [ ]:
pred_col = 'hospital'

df_num[pred_col].value_counts()

np.random.seed(1)

dataset_class_1 = df_num[df_num[pred_col]==1]
dataset_class_0 = df_num[df_num[pred_col]==0]

# selecting a subpopulation of X% that will be used for training and testing
n_subpop = int(1*df_num.shape[0])
i_class_yes = np.random.randint(0, high=dataset_class_1.shape[0], size=n_subpop)
i_class_no = np.random.randint(0, high=dataset_class_0.shape[0], size=n_subpop)

X = dataset_class_1.iloc[i_class_yes].append(dataset_class_0.iloc[i_class_no])
y = X[pred_col]
X_feat = X.drop(columns=[pred_col])
feature_names = X_feat.columns
iscat = [x in categorical_feat for x in feature_names]

X_train, X_test, y_train, y_test = train_test_split(X_feat, y, test_size=0.33, random_state=42)

X_test_nd = X_test.to_numpy(dtype=float)
X_train_nd = X_train.to_numpy(dtype=float)

# Normalizing data ********* <<< DEBUG, remove next rows if you don't want normalization >>> **********
scaler = MinMaxScaler()

X_train_nd = scaler.fit_transform(X_train_nd)
X_train = pd.DataFrame(X_train_nd, columns=feature_names)

X_test_nd = scaler.fit_transform(X_test_nd)
X_test = pd.DataFrame(X_test_nd, columns=feature_names)

## **Classifiers**

In [ ]:
#CLASSIFIERS
def get_best_threshold_f1(X,y,y_pred):
  prec, recall, threshold = precision_recall_curve(y, y_pred)
  f1 = 2*prec*recall/(prec+recall)
  return(threshold[np.where(f1==np.nanmax(f1))])

print("True Labels")
print(collections.Counter(y_test))
print()

#Gradient Boosting
clf_gb = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0).fit(X_train, y_train)
ypred_proba = clf_gb.predict_proba(X_test)[:,1]
sel_thr_gb = get_best_threshold_f1(X_test,y_test,ypred_proba)
ypred_gb = (clf_gb.predict_proba(X_test)[:,1] >= sel_thr_gb).astype(int)

print("Gradient Boosting")
print(collections.Counter(ypred_gb))
print(f"F1: {f1_score(y_test, ypred_gb)}")

#Random Forest
clf_rf = RandomForestClassifier(random_state=1).fit(X_train, y_train)
ypred_proba = clf_rf.predict_proba(X_test)[:,1]
sel_thr_rf = get_best_threshold_f1(X_test,y_test,ypred_proba)
ypred_rf = (clf_rf.predict_proba(X_test)[:,1] >= sel_thr_rf).astype(int)

print("Random Forest")
print(collections.Counter(ypred_rf))
print(f"F1: {f1_score(y_test, ypred_rf)}")

#Logistic Regression
clf_lr = LogisticRegression(random_state=1, penalty='l1', solver='liblinear').fit(X_train, y_train)
ypred_proba = clf_lr.predict_proba(X_test)[:,1]
sel_thr_lr = get_best_threshold_f1(X_test,y_test,ypred_proba)
ypred_lr = (clf_lr.predict_proba(X_test)[:,1] >= sel_thr_lr).astype(int)

print("Logistic Regression")
print(collections.Counter(ypred_lr))
print(f"F1: {f1_score(y_test, ypred_lr, pos_label=0)}")

#Multi-layer perceptron (neural network)
clf_nn = MLPClassifier(random_state=1,hidden_layer_sizes=(100, 50, 20), max_iter=500).fit(X_train, y_train)
ypred_proba = clf_nn.predict_proba(X_test)[:,1]
sel_thr_nn = get_best_threshold_f1(X_test,y_test,ypred_proba)
ypred_nn = (clf_nn.predict_proba(X_test)[:,1] >= sel_thr_nn).astype(int)

print("Multi-layer Perceptron")
print(collections.Counter(ypred_nn))
print(f"F1: {f1_score(y_test, ypred_nn, pos_label=0)}")

classifier_dict = {'NN': clf_nn, 'GB': clf_gb, 'LR': clf_lr, 'RF': clf_rf}
classifier_thr = {'NN': sel_thr_nn, 'GB': sel_thr_gb, 'LR': sel_thr_lr, 'RF': sel_thr_rf}

print("Thresholds")
print(classifier_thr)

## **Explainers: Overall Metrics**

### **Araucana**

In [ ]:
#ARAUCANA EXPLAINER

def run_auracana(X_test, feature_names, cat_list, X_train, nsize, oversampling, oversampling_size, classifiers):
    xai_dict = dict(zip(list(classifiers.keys()), [[], [], [], []]))
    fidelity_dict = {}
    times = {}
    # for each classifier
    for k_clf, v_clf in classifiers.items():
        xai_start = time.time()
        fidelity_list = []
        fidelity_fail_count = 0
        # for each instance of the test set
        for i in tqdm(range(X_test.shape[0])):
            # declare the instance from the test set we want to explain
            instance = X_test[i, :].reshape(1, X_test.shape[1])
            # save the predicted label for the instance according to the current classifier
            instance_pred_y = (v_clf.predict_proba(instance)[:,1] >= classifier_thr[k_clf]).astype(int)
            # build xai tree to explain the instance classification
            try:
                xai_tree = araucanaxai.run(x_target=instance, y_pred_target=instance_pred_y,
                                           x_train=X_train, feature_names=feature_names,
                                           cat_list=cat_list, oversampling=oversampling, oversampling_size = oversampling_size,
                                           predict_fun= lambda instance:(v_clf.predict_proba(instance)[:,1] >= classifier_thr[k_clf]).astype(int), 
                                           neighbourhood_size=nsize)
                xai_dict[k_clf].append(xai_tree)
                #fidelity online check 
                if xai_tree['tree'].predict(instance) != instance_pred_y:
                  fidelity_fail_count += 1
                  if fidelity_fail_count == 100:
                    warnings.warn("Warning...........divergent predictions between explainer and original model")
                fidelity_list.append((xai_tree['tree'].predict(instance) == instance_pred_y).astype(int))
            except Exception as e:
                print(e)
                xai_dict[k_clf].append(None)
                fidelity_list.append(None)
                continue
        if len([x for x in fidelity_list if x != None]) != 0:
            fidelity_dict[k_clf] = sum([x for x in fidelity_list if x != None]) / len([x for x in fidelity_list if x != None])
        else:
            fidelity_dict[k_clf] = None
        times[k_clf]=time.time() - xai_start
    return {"fidelity": fidelity_dict, "test": xai_dict, "time": times}

In [ ]:
# Function to compare explaination
def exp_similarity_trees(tree_a, tree_b):
    feat_a = tree_a.tree_.feature.copy()
    feat_b = tree_b.tree_.feature.copy()
    n = min(len(feat_a), len(feat_b))       # if different len, trim the longer
    feat_a = feat_a[0:n]
    feat_b = feat_b[0:n]
    return sum((feat_a == feat_b).astype(int)) / n

# FIDELITY: concordance of the predictions between the XAI proxy model and the complex model

def check_fidelity(araucana_xai):
  for c in classifier_dict.keys():
    print('- Classifier ', c, ': fidelity = ', araucana_xai["fidelity"][c])

# IDENTITY: if there are 2 identical instances, they must have identical explanations

def check_identity(X_test, araucana_xai):
  iDup = np.where(X_test.duplicated(keep=False))[0]  # instances in test which are duplicated
  xai_dup = []
  dup = []
  for couple in list(combinations(iDup, 2)):
    check = (X_test.iloc[couple[0],] == X_test.iloc[couple[1],]).all()
    if check:
      dup.append(couple)
  for c in classifier_dict.keys():
    if len(dup)!=0:
      right = 0
      for d in dup:
        xai1 = araucana_xai["test"][c][d[0]]['tree']
        xai2 = araucana_xai["test"][c][d[1]]['tree']
        s = exp_similarity_trees(xai1, xai2)
        if s==1:
          right += 1
      score = right/len(dup)
      print('- Classifier', c, ': identity = ', score)
    else:
      print('- Classifier', c, ': identity = na')

# SEPARABILITY: if there are 2 dissimilar instances, they must have dissimilar explanations

def check_separability(X_test_nd, araucana_xai):
  iNotDup = [i for i in range(len(X_test_nd)) if i not in iDup] # instances in test which are NOT duplicated
  combs = list(combinations(iNotDup, 2))
  for c in classifier_dict.keys():
    wrong = 0
    for couple in combs:
        xai1 = araucana_xai["test"][c][couple[0]]['tree']
        xai2 = araucana_xai["test"][c][couple[1]]['tree']
        s = exp_similarity_trees(xai1, xai2)
        if s==1:
            wrong += 1
    total = len(combs)
    score = wrong/total
    print('- Classifier', c, ': separability = ', score, "(",wrong, 'equals out of',total,")")

#### Single Run

In [ ]:
os_method = "smote" # Possible values = [None,"smote", "uniform", "non-uniform"]

#following parameters are not used if os_method = None
nratio = 0.15 # neighborhood size expressed as a portion of training size
os_size = 200 # number of new samples to be generated with oversampling
nsize = int(X_train.shape[0] * nratio)

araucana_xai = run_auracana(X_test=X_test_nd, cat_list=iscat, feature_names=feature_names,
                            X_train=X_train_nd, nsize=nsize, oversampling=os_method, oversampling_size = os_size,
                            classifiers = classifier_dict)

print('\ntime')
for c in classifier_dict.keys():
  print('- Classifier ', c, ': time = ', araucana_xai["time"][c])

print('\nfidelity')
check_fidelity(araucana_xai)

print('\nidentity')
check_identity(X_test, araucana_xai)

print('\nseparability')
check_separability(X_test_nd, araucana_xai)

#### Hyperaparameter Search

Oversampling Methods

In [ ]:
os_methods = ["smote", "uniform", "non-uniform"]
os_size = 100 # number of new samples to be generated with oversampling
nsize_dict = {"smote": 800, "uniform":35, "non-uniform":35}

for os_method in os_methods:
  print(str(os_method))
  print(nsize_dict[os_method])
  araucana_xai = run_auracana(X_test=X_test_nd, cat_list=iscat, feature_names=feature_names,
                              X_train=X_train_nd, nsize=nsize_dict[os_method], oversampling=os_method, oversampling_size = os_size,
                              classifiers = classifier_dict)
  # TIME: 
  print('\ntime')
  for c in classifier_dict.keys():
    print('- Classifier ', c, ': time = ', araucana_xai["time"][c])

  # FIDELITY: concordance of the predictions between the XAI proxy model and the complex model
  print('\nfidelity')
  for c in classifier_dict.keys():
    print('- Classifier ', c, ': fidelity = ', araucana_xai["fidelity"][c])

  # IDENTITY: if there are 2 identical instances, they must have identical explanations
  print('\nidentity')
  check_identity(X_test, araucana_xai)

  # SEPARABILITY: if there are 2 dissimilar instances, they must have dissimilar explanations
  print('\nseparability')
  check_separability(X_test_nd, araucana_xai)

Oversampling Neighborhood Size

In [ ]:
nratios = [0.25, 0.10, 0.05, 0.01, 0.005] # neighborhood size expressed as a portion of training size
os_method = "non-uniform" # Possible values = [None,"smote", "uniform", "non-uniform"]
os_size = 50 # number of new samples to be generated with oversampling

for nratio in nratios:
  nsize = int(X_train.shape[0] * nratio)
  print(nsize)
  araucana_xai = run_auracana(X_test=X_test_nd, cat_list=iscat, feature_names=feature_names,
                              X_train=X_train_nd, nsize=nsize, oversampling=os_method, oversampling_size = os_size,
                              classifiers = classifier_dict)
  # TIME: 
  print('\ntime')
  for c in classifier_dict.keys():
    print('- Classifier ', c, ': time = ', araucana_xai["time"][c])

  # FIDELITY: concordance of the predictions between the XAI proxy model and the complex model
  print('\nfidelity')
  for c in classifier_dict.keys():
    print('- Classifier ', c, ': fidelity = ', araucana_xai["fidelity"][c])

  # IDENTITY: if there are 2 identical instances, they must have identical explanations
  print('\nidentity')
  check_identity(X_test, araucana_xai)

  # SEPARABILITY: if there are 2 dissimilar instances, they must have dissimilar explanations
  print('\nseparability')
  check_separability(X_test_nd, araucana_xai)

Oversampling Sample Size

In [ ]:
nratio = 0.01 # neighborhood size expressed as a portion of training size
os_method = "non-uniform" # Possible values = [None,"smote", "uniform", "non-uniform"]
os_sizes = [1, 10, 50, 100, 500] # number of new samples to be generated with oversampling
nsize = int(X_train.shape[0] * nratio)

for os_size in os_sizes:
  print(os_size)
  araucana_xai = run_auracana(X_test=X_test_nd, cat_list=iscat, feature_names=feature_names,
                              X_train=X_train_nd, nsize=nsize, oversampling=os_method, oversampling_size = os_size,
                              classifiers = classifier_dict)
  # TIME: 
  print('\ntime')
  for c in classifier_dict.keys():
    print('- Classifier ', c, ': time = ', araucana_xai["time"][c])

  # FIDELITY: concordance of the predictions between the XAI proxy model and the complex model
  print('\nfidelity')
  for c in classifier_dict.keys():
    print('- Classifier ', c, ': fidelity = ', araucana_xai["fidelity"][c])

  # IDENTITY: if there are 2 identical instances, they must have identical explanations
  print('\nidentity')
  check_identity(X_test, araucana_xai)

  # SEPARABILITY: if there are 2 dissimilar instances, they must have dissimilar explanations
  print('\nseparability')
  check_separability(X_test_nd, araucana_xai)

### **SHAP**

In [ ]:
#SHAP EXPLAINER

def run_shap(X_test, classifiers):
    xai_dict = dict(zip(list(classifiers.keys()), [[], [], []])) 
    fidelity_dict = {}
    times = {}
    # for each classifier
    for k, v in classifiers.items():
      start = time.time()
      #For Logistic Regression, we need to use shap.Explainer
      if k=='LR' or k=='NN':
        explainer = shap.Explainer(v.predict_proba, np.median(X_test, axis=0).reshape(1, X_test.shape[1]))
        shap_val = explainer(X_test) 
        shap_values = shap_val.values
      #For Random Forest and Gradient Boosting, we can use shap.TreeExplainer
      else:
        explainer = shap.TreeExplainer(v)
        shap_values = explainer.shap_values(X_test)
      if k=='RF':
        shap_values = shap_values[1]
      xai_dict[k] = shap_values
      times[k] = time.time()-start
      fidelity_dict[k] = 1 #by default, not a local explainer
    return {"fidelity": fidelity_dict, "test": xai_dict, "time": times}

shap_xai = run_shap(X_test=X_test_nd, classifiers = classifier_dict)

print('\ntime')
for c in classifier_dict.keys():
  print('- Classifier ', c, ': time = ', shap_xai["time"][c])

In [ ]:
for c in classifier_dict.keys():
  print('- Classifier ', c, ': time = ', shap_xai["time"][c])

In [ ]:
# Function to compare explaination (oversimplified, we only need to check if they are equal or not at the moment)
def exp_similarity_shap(shap_vals_a, shap_vals_b):
    #EXACT MATCH VALUE BASED
    #is_equal = np.array_equal(shap_vals_a,shap_vals_b)
    #EXACT MATCH FEATURE RANKING
    if len(shap_vals_a.shape)>1:
      shap_vals_a = shap_vals_a[:,1]
    z1 = zip(feature_names, abs(shap_vals_a))
    z1 = list(z1)
    z1 = sorted(z1, key = lambda x: x[1])
    z1 = [z1[i][0] for i in range(len(z1))]

    if len(shap_vals_b.shape)>1:
      shap_vals_b = shap_vals_b[:,1]
    z2 = zip(feature_names, abs(shap_vals_b))
    z2 = list(z2)
    z2 = sorted(z2, key = lambda x: x[1])
    z2 = [z2[i][0] for i in range(len(z2))]
    return 1 if z1==z2 else 0

# FIDELITY (1 by default, not a local explainer)

print('\nfidelity')
for c in classifier_dict.keys():
  print('- Classifier ', c, ': fidelity = ', shap_xai["fidelity"][c])

# IDENTITY

print('\nidentity')
iDup = np.where(X_test.duplicated(keep=False))[0]  # instances in test which are duplicated
xai_dup = []
dup = []
for couple in list(combinations(iDup, 2)):
  check = (X_test.iloc[couple[0],] == X_test.iloc[couple[1],]).all()
  if check:
    dup.append(couple)

for c in classifier_dict.keys():
  if len(dup)!=0:
    right = 0
    for d in dup:
      s = exp_similarity_shap(shap_xai["test"][c][d[0]], shap_xai["test"][c][d[1]])
      if s==1:
        right += 1
    score = right/len(dup)
    print('- Classifier', c, ': identity = ', score)
  else:
   print('- Classifier', c, ': identity = na')

# SEPARABILITY

print('\nseparability')
combs = list(combinations(range(len(X_test_nd)), 2))
for c in classifier_dict.keys():
  wrong = 0
  for couple in combs:
      s = exp_similarity_shap(shap_xai["test"][c][couple[0]], shap_xai["test"][c][couple[1]])
      if s==1:
          wrong += 1
  total = len(combs)
  score = wrong/total
  print('- Classifier', c, ': separability = ', score, "(",wrong, 'equals out of',total,")")

### **LIME**

In [ ]:
###DEBUG
def return_weights(exp):
   
    exp_list = exp.as_map()[1]
    #print(exp_list) = [(7, -0.539), (58, 0.215), etc]
    exp_list = sorted(exp_list, key=lambda x: x[0]) 
    #print(exp_list) = [(0, 0.014), (1, -0.061), (2, 0.046), etc]
    #sort so that we can compare with feature list (0 = feature at position 0 in the feature list)
    exp_weight = [x[1] for x in exp_list]
    
    return exp_weight

explainer = lime.lime_tabular.LimeTabularExplainer(X_train_nd, 
                                                    feature_names=feature_names, 
                                                    verbose=False, 
                                                    mode='classification')

for k, v in classifier_dict.items():
  print(f"\n\nClassifier: {k}")
  trs = classifier_thr[k]

  weights = []
  fidelity = 0

  #Iterate over rows in feature matrix
  for i in tqdm(range(len(X_test_nd))):
      
      #Get explanation for ith row
      exp = explainer.explain_instance(X_test_nd[i, :], 
                                      v.predict_proba,
                                      num_features = X_test_nd.shape[1],
                                      num_samples = 200)
      #Fidelity check
      instance = X_test_nd[i, :].reshape(1, X_test_nd.shape[1])
      instance_pred_y = (v.predict_proba(instance)[:,1] >= trs).astype(int)
      if (exp.predict_proba[1] >= 0.5).astype(int) == instance_pred_y:
        fidelity +=1

      #Get weights
      exp_weight = return_weights(exp)
      weights.append(exp_weight)
      
  #Create DataFrame
  lime_weights = pd.DataFrame(data=weights,columns=feature_names)
  display(lime_weights)

  #Fidelity
  print(f"\nFidelity = {fidelity/len(X_test_nd)}")

  #Identity
  iDup = np.where(X_test.duplicated(keep=False))[0]  # instances in test which are duplicated
  couples = list(combinations(iDup, 2))
  equal_count = 0 
  couples = random.sample(couples, int(0.5*len(couples))) ###DEBUG
  for couple in tqdm(couples):
    check = (X_test.iloc[couple[0],] == X_test.iloc[couple[1],]).all()
    if check:
      exp_names1 = abs(lime_weights.iloc[couple[0]]).sort_values(ascending=False).index.tolist()
      exp_names2 = abs(lime_weights.iloc[couple[1]]).sort_values(ascending=False).index.tolist()
      if exp_names1==exp_names2:
        equal_count += 1
  if len(couples)==0:
    print("\nIdentity = n.a.")
  else: 
    print(f"\nIdentity = {equal_count/len(couples)}")

  #Separability
  iNotDup = [i for i in range(len(X_test_nd)) if i not in iDup] # instances in test which are NOT duplicated
  couples = list(combinations(iNotDup, 2))
  equal_count = 0
  couples = random.sample(couples, int(0.25*len(couples))) ###DEBUG
  for couple in tqdm(couples):
    exp_names1 = abs(lime_weights.iloc[couple[0]]).sort_values(ascending=False).index.tolist()
    exp_names2 = abs(lime_weights.iloc[couple[1]]).sort_values(ascending=False).index.tolist()
    if exp_names1==exp_names2:
      equal_count += 1

  print(f"\nSeparability = {equal_count/len(couples)}")

## Explainers: Single Instance (i.e. Local) Explanations

In [ ]:
# declare the classifier
k_clf = "NN"
v_clf = classifier_dict[k_clf]
# it could be usefult to separate right predictions and wrong predictions
right = np.where((y_test-(v_clf.predict_proba(X_test)[:,1] >= classifier_thr[k_clf]).astype(int) == 0))[0]
wrong = np.where((y_test-(v_clf.predict_proba(X_test)[:,1] >= classifier_thr[k_clf]).astype(int) != 0) & (y_test==0))[0]

In [ ]:
# declare the instance from the test set we want to explain
ind = wrong[0]
instance = X_test_nd[ind, :].reshape(1, X_test.shape[1])

# save the predicted label for the instance according to the current classifier
instance_pred_y = (v_clf.predict_proba(instance)[:,1] >= classifier_thr[k_clf]).astype(int)

# cumbersome line to print the example nicely
print(pd.DataFrame(data=[np.concatenate([instance.flatten(),instance_pred_y],dtype=object)],
                     columns=np.concatenate([feature_names,["pred_label"]],dtype=object)))

print(f"Predicted class:{instance_pred_y}")
print(f"True class:{y_test.iloc[ind]}")

## ARAUCANA
print("\n\n ==================== ARAUCANA ====================\n\n")

# build xai tree to explain the instance classification
xai_tree = araucanaxai.run(x_target=instance, y_pred_target=instance_pred_y,
                           x_train=X_train_nd, feature_names=feature_names,
                           cat_list=iscat, oversampling=None, max_depth=4, neighbourhood_size=300,
                           predict_fun= v_clf.predict)
from sklearn import tree
fig, ax = plt.subplots(figsize=(10, 10))
tree.plot_tree(xai_tree['tree'], feature_names=feature_names, filled=True, class_names=["None","Death"])
plt.tight_layout()
plt.show()

## SHAP
print("\n\n ==================== SHAP ====================\n\n")

explainer = shap.Explainer(v_clf.predict_proba, np.median(X_test, axis=0).reshape(1, X_test.shape[1]))
shap_val = explainer(instance) 
base_values = shap_val.base_values
shap_values = shap_val.values

labels = feature_names
x = np.arange(len(labels))
w=0.9
fig, ax = plt.subplots(figsize=(8,10))
vals = shap_val[0].values[:,1]
bars = plt.barh(x, vals, label='SHAP', height=w, color= ['#449F89' if i else '#CA7952' for i in vals > 0])
plt.axvline(x=0, ymin=0, ymax=1, color='grey')
ax.set_yticks(x)
ax.set_yticklabels(labels, fontsize=12)
plt.title('SHAP values')
plt.show()

## LIME
print("\n\n ==================== LIME ====================\n\n")

explainer_lime = lime.lime_tabular.LimeTabularExplainer(X_train_nd, feature_names = feature_names)
exp = explainer_lime.explain_instance(data_row=X_test_nd[ind, :], 
                                      predict_fn=v_clf.predict_proba,
                                      num_features=X.shape[1],                # ********* <<< DEBUG, replace with X.shape[1] >>> **********
                                      top_labels=1)

exp_list = exp.as_list()
vals = [x[1] for x in exp_list]
names = [x[0] for x in exp_list]
vals.reverse()
names.reverse()
colors = ['#4986E7' if x > 0 else '#D93025' for x in vals]
x = np.arange(len(labels))
w=0.9
fig, ax = plt.subplots(figsize=(8,10))
bars = plt.barh(x, vals, label='LIME', height=w, color= colors)
plt.axvline(x=0, ymin=0, ymax=1, color='grey')
ax.set_yticks(x)
ax.set_yticklabels(names, fontsize=12)
plt.title('LIME')
plt.show()